In [0]:
#Import the necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

np.set_printoptions(suppress=True)

## Load the cifar dataset

In [2]:
#download data
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.cifar10.load_data()

#labels=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

#labels 0,1,8,9 --->  Transport
#labels 2,3,4,5,6,7 ----> Animals
#For training data combine the 10 classes to two classes,0 for transport and 1 for animals
y_train[np.where(np.isin(y_train,[0,1,8,9]))] = 0
y_train[np.where(np.isin(y_train,[2,3,4,5,6,7]))] = 1

#two classes labels
labels = ['Transport','Animal']
#df1=pd.DataFrame(y_train, columns=['X'])
#df1.X.value_counts()

#For test data combine the 10 classes to two classes,0 for transport and 1 for animals
y_test[np.where(np.isin(y_test,[0,1,8,9]))] = 0
y_test[np.where(np.isin(y_test,[2,3,4,5,6,7]))] = 1
x_train.shape
y_train.shape

170500096/170498071 [==============================] - 2s 0us/step


(50000, 1)

In [3]:
#Install the library for using tensorboard in the colab environment
!pip install tensorboardcolab

In [4]:
#Load the library and create a writer that will output information for tensorboard
import tensorboardcolab

Using TensorFlow backend.


In [5]:
tbc = tensorboardcolab.TensorBoardColab()

writer = tbc.get_writer()

Wait for 8 seconds...


W0720 20:53:26.412325 140582032541568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tensorboardcolab/core.py:49: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



TensorBoard link:
https://5b47d21d.ngrok.io


## Normalize the inputs and reshape the labels

In [6]:
#reshape training labels to one-hot coding
y_input = tf.keras.utils.to_categorical(y_train[:40000])

#normalise the training data
x_input = (x_train[:40000]  / 255.0)

#subset validatuon data and reshape validation labels to one-hot coding
y_valid = tf.keras.utils.to_categorical(y_train[40000:])
x_valid = (x_train[40000:] / 255.0)

#reshape test data
x_test_input = x_test / 255.0

40000

In [0]:
#function to divide the data into batches for training
data_index = 0
def generate_batch(batch_size):
    global data_index
    batch = np.ndarray(shape=(batch_size, 32, 32, 3), dtype=np.float32)  #the same shapes as train data
    labels = np.ndarray(shape=(batch_size,2), dtype=np.float32)
    for i in range(batch_size):
        batch[i] = x_input[data_index]
        labels[i] = y_input[data_index]
        data_index = (data_index + 1) % len(x_input)
    return batch, labels

## Create the placeholders

In [0]:
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 2])

## Now we'll build the convolutional layer

In [9]:
# input Layer
conv_layer = tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3),activation=tf.nn.relu)(x)

#Flatten layer

flat_layer = tf.keras.layers.Flatten()(conv_layer)

#Fully connected layer
dense = tf.keras.layers.Dense(units=512, activation=tf.nn.relu)(flat_layer)

# output layer using softmax activation function
y = tf.keras.layers.Dense(units=2,activation=tf.nn.softmax)(dense)

W0720 20:53:27.377913 140582032541568 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## Create the loss

In [0]:
#crossentropy loss function
cost = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

## Set up the training parameters

In [0]:
start_lr = 1e-10
global_step = tf.Variable(0,trainable=False)
lr = tf.train.exponential_decay(start_lr, global_step=global_step,decay_steps=100, decay_rate=1.30)
optimizer= tf.train.AdamOptimizer(lr) 
gd_train =optimizer.minimize(cost,global_step=global_step)

In [0]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
# to calculate the accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [0]:
#Write out the graph to tensorboard
sess = tf.Session()
writer.add_graph(sess.graph)
sess.close()

In [15]:
#we'll look at training/validation accuracy and training/validation loss as it changes during training
#TensorBoard summary for the validation accuracy
val_acc = tf.summary.scalar(name='val_acc', tensor=accuracy)

#TensorBoard summary for the train accuracy
train_acc = tf.summary.scalar(name='train_acc', tensor=accuracy)

#TensorBoard summary for the validation loss
loss_val = tf.summary.scalar(name='loss_val', tensor=cost)

#TensorBoard summary for the train loss
loss_train = tf.summary.scalar(name='loss_train', tensor=cost)

#TensorBoard summary for the learning rate
learning_rate = tf.summary.scalar(name="learning_rate",tensor=lr)
tf.summary.scalar("current_step", global_step)


<tf.Tensor 'current_step:0' shape=() dtype=string>

## Create a session and run the training

In [16]:
#Number of epochs and batch size
epochs=12000
batch_size= 128

#Global variables initializer
init = tf.global_variables_initializer()

#Starting the Tensorflow session
sess = tf.Session()

#Initializing the Variables
sess.run(init)

#Iterating through all the epochs
for i in range(epochs):
  
  #increment global step for exponential decau learning rate
  global_step = tf.train.get_global_step()
  
  # generate data batches
  batch_x, batch_y = generate_batch(batch_size)
  
  #Feeding data into optimizer and calculate the training loss function and training accuracy
  _,train_loss, train_acc = sess.run([gd_train,loss_train,train_accuracy], feed_dict={x:batch_x, y_:batch_y})
      
  #Feeding data into optimizer and calculate the validation loss function and validation accuracy
  val_acc,val_loss= sess.run([val_accuracy,loss_val], feed_dict = { x: x_valid, y_: y_valid})
  
  rt= sess.run(learning_rate)
  
  # write scalars to tensorbaord
  writer.add_summary(train_acc,i)
  writer.add_summary(val_acc,i)
  writer.add_summary(val_loss,i)
  writer.add_summary(train_loss,i)
  writer.add_summary(rt,i)
  #print(sess.run(optimizer._lr))
  if i%1000 == 0:
    print('Training Step:' + str(i) + '  valid_Loss = ' + str(sess.run(cost, {x: x_valid, y_: y_valid})) +  '  train_Loss = ' + str(sess.run(cost, {x: batch_x, y_: batch_y})))
    

Training Step:0  valid_Loss = 0.7287667  train_Loss = 0.7294983
Training Step:1000  valid_Loss = 0.7282753  train_Loss = 0.7635807
Training Step:2000  valid_Loss = 0.720246  train_Loss = 0.7507807
Training Step:3000  valid_Loss = 0.65772015  train_Loss = 0.67588913
Training Step:4000  valid_Loss = 0.447682  train_Loss = 0.39805686
Training Step:5000  valid_Loss = 0.26390812  train_Loss = 0.26648942
Training Step:6000  valid_Loss = 0.24621046  train_Loss = 0.22421446
Training Step:7000  valid_Loss = 0.3387335  train_Loss = 0.17603448
Training Step:8000  valid_Loss = 0.67597747  train_Loss = 0.69491243
Training Step:9000  valid_Loss = nan  train_Loss = nan
Training Step:10000  valid_Loss = nan  train_Loss = nan
Training Step:11000  valid_Loss = nan  train_Loss = nan


In [17]:
#Evaluate the model by calculating the test data accuracy and print it
y_prob = sess.run(y, feed_dict={x: x_test_input})

print(y_prob.shape)
y_pred = np.argmax(y_prob,axis=-1)
print('test accuracy: ', np.sum(y_pred==y_test[:,0])/len(y_test))

(10000, 2)
test accuracy:  0.4


In [0]:
sess.close()